# Notebook 2: *Gaussian Processes* (part 1)

Probabilistic Machine Learning -- Spring 2023, UniTS

<a target="_blank" href="https://colab.research.google.com/github/emaballarin/probml-units/blob/main/notebooks/02a_gaussian_processes_mle.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

***Gaussian Processes*** (GPs) are *stochastic processes*, *i.e.* collections of random variables, such that every finite collection of such has a multivariate normal distribution. 
GPs can be interpreted as ***distributions over functions*** $f(x)$. In fact, whenever we observe a finite set of values $(f(x_1),\ldots,f(x_N))$, we assume they are jointly normally distributed with some mean $\mu(x)$ and covariance $\Sigma(x)$. Let

$$ f(x) \sim \mathcal{GP}\big(\mu(x), \Sigma(x)\big)$$

denote a Gaussian Process, where for any finite subset $\bar{x}=(x_1,\ldots,x_N)$, the marginal distribution is a multivariate Gaussian $ f(\bar{x})\sim \mathcal{N}\big(\mu(\bar{x}), \Sigma(\bar{x})\big).$

## Kernels

The covariance matrix $\Sigma(x)$ of a GP is defined by a ***kernel function*** $k$. To ensure that the learned function preserves similarity between the inputs, it is required that $ \Sigma_{ij}=k(x_i,x_j). $

A kernel (or covariance function) $k:\mathcal{X}\times\mathcal{X}\rightarrow \mathbb{R}$ defines the statistical relationship between two variables in a space $\mathcal{X}$. It is symmetric and non-negative, meaning that 

\begin{align*}
k(x,x')&=k(x',x)\\
k(x,x')&\geq0
\end{align*}

so it acts as a measure of similarity in $\mathcal{X}$ and the resulting matrix $\Sigma(x)$ is always symmetric and positive definite.

The choice of a specific kernel function allows to set a prior distribution over functions.

Examples of valid kernels are *e.g.* the ***radial basis function*** (RBF) kernel

$$
k(x,x')=\sigma^2 \exp \Bigg(-\frac{||x-x'||^2}{2 l^2}\Bigg),
$$

the (uniform) ***polynomial*** kernel 

$$
k(x,x')=\sigma^2 (x^T x'+b)^d
$$

and the **periodic** kernel 

$$ k(x,x')=\sigma^2 \exp\Bigg( -\frac{2 \sin^2 (\pi (x-x')/p)}{l^2} \Bigg).
$$

![](./img/gp_mle_1.png)
<br><sub><sup>Example of covariance matrix of kernels computed on 50 equispaced points (left) and 3 samples drawn from a Normal distribution with such covariance and zero mean (right). Kernel parameters: (sigma = 1; l = 1; b = 0.5; p = 3) </a></sup></sub>

Kernel functions can also be combined together to generate more complex kernels, by using a set of suitable operations that preserve kernel properties. E.g. we can use products and linear combinations of kernels. 

![](./img/gp_mle_2.png)
<br><sub><sup>Examples of composition of kernels (same parameters as before)</a></sup></sub>


## *GP regression*

We want to model a dependent variable $y$ as a function of an independent variable $x$, using a set of observations $D=\{(x_i,y_i)\}_i$. 

Classical Bayesian regression fits a parametric function $f(\theta,\cdot)$ to the data, i.e. it infers a probability distribution $f(\theta|D)$. Rather than learning single point estimates of $\theta$, it places a prior over parameters and updates the distribution whenever new data points are observed.

GPs, instead, have a ***non-parametric*** approach that infers a distribution $p(f|D)$ over all possible functions $f$. 

### Case: Noise-free observations

Consider a 1-dimensional GP with mean $\mu=\mathbf{0}$ and an RBF kernel with $\sigma=1$ and $l=0.2$.

Given a test set $X_*=x_1,\ldots,x_N$, we want to predict the outputs $f_*=(f(x_1),\ldots,f(x_N))$.
Before observing any training data, we can sample from the prior distribution on $X_*$

$$
f_*\sim\mathcal{N}(\mathbf{0},K_{**}),
$$

where $K_{**}=k(X_*, X_*)$.

![](./img/gp_mle_3.png)
<br><sub><sup>Plot of 3 samples from the GP prior, defined on the same equispaced points (sigma = 1; l = 0.2)</a></sup></sub>

Suppose we observe a training set $X=\{(x_i, y_i)\}_i$, where the values of $y_i=f(x_i)$ are exact (i.e. non-noisy). The **joint distribution** on training and test points has the following form

$$
\binom{f}{f_*} \sim \mathcal{N} \Bigg(\binom{\mu}{\mu_*} , 
\begin{pmatrix} K & K_* \\ K_*^T & K_{**} \end{pmatrix}  \Bigg),
$$

where $K=k(X,X)$ and $K_*=k(X,X_*)$.


Now we can ***condition* on the training set** $X$ and obtain posterior mean and posterior covariance:

\begin{align*}
f_*|X_*,X,f &\sim \mathcal{N}(\mu_*,\Sigma_*)\\
\mu_* &= \mu(X_*) + K_*^T K^{-1} (f-\mu(X))\\
\Sigma_* &= K_{**}-K_*^T K^{-1} K_*.
\end{align*}

![](./img/gp_mle_4.png)
<br><sub><sup>Noise-free regime. 3 samples from the GP, after conditioning on some training points (blue dots). The mean (dashed red line) and 1-standard-deviation credibility interval (transparent fill) are also shown.</a></sup></sub>

Notice that since the observations are noiseless, the GP acts as an **interpolator** of the training data.

### Case: Noisy observations

Now suppose that the observations are noisy 
$$y_i=f(x_i)+\epsilon \qquad \epsilon \sim\mathcal{N}(0, \sigma^2).$$

![](./img/gp_mle_5.png)
<br><sub><sup>Noisy regime. 3 samples from the GP, after conditioning on some training points (blue dots): the same points as before, with added Gaussian homoskedastic (sigma = 0.5) noise. The mean (dashed red line) and 1-standard-deviation credibility interval (transparent fill) are also shown.</a></sup></sub>

In this case, the model cannot interpolate the training points, but only get closer to them after conditing. By changing the kernel hyperparameters we could obtain a significantly different fit to the data.

![](./img/gp_mle_6a.png)
![](./img/gp_mle_6b.png)
<br><sub><sup>Effect of different hyperparameter choices on the learned kernel. Training data is the same as before. Above: sigma = 0.3, l = 0.2; Below: sigma = 1, l = 0.5 </a></sup></sub>

### Learning kernel (hyper)parameters

Instead of arbitrarily choosing the kernel hyperparameters, we could infer them directly from the observed data. *E.g.* we could optimize the kernel hyperparameters $\theta$ by maximizing the marginal log-likelihood $ p(y| x,\theta)$ on the observed data $(x,y)$

$$
\hat{\theta} = \text{argmax}_\theta p(y| x,\theta).
$$

![](./img/gp_mle_7.png)
<br><sub><sup>3 samples from the GP, after conditioning on some training points (blue dots): the same points as before, with added Gaussian homoskedastic (sigma = 0.5) noise. The mean (dashed red line) and 1-standard-deviation credibility interval (transparent fill) are also shown. Kernel hyperparameters fitted by MLL of the data.</a></sup></sub>

## References:
[1] K. P. Murphy, ["Machine Learning: A Probabilistic Perspective"](http://noiselab.ucsd.edu/ECE228/Murphy_Machine_Learning.pdf) (Chap. 15)

[2] C. M. Bishop, ["Pattern Recognition and Machine Learning"](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf) (Chap. 6)

[3] [SKlearn GPs documentation](https://scikit-learn.org/stable/modules/gaussian_process.html)